In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!rm Gogo_Shogi.so
!rm gk2_for_import.o
!rm wrap_file.o
!ls 
!echo
!gcc -fPIC -c -o gk2_for_import.o drive/MyDrive/Gogo_Shogi/gk2_for_import.c -lpython3.8  -I/usr/local/lib/python3.8/dist-packages/tensorflow/include/external/local_config_python/python_include
!gcc -fPIC -c -o wrap_file.o drive/MyDrive/Gogo_Shogi/wrap_file.c -lpython3.8 -I/usr/local/lib/python3.8/dist-packages/tensorflow/include/external/local_config_python/python_include
!gcc -fPIC -lpython3.8 -shared -o Gogo_Shogi.so gk2_for_import.o wrap_file.o 
!ls -a

rm: cannot remove 'Gogo_Shogi.so': No such file or directory
rm: cannot remove 'gk2_for_import.o': No such file or directory
rm: cannot remove 'wrap_file.o': No such file or directory
drive  sample_data

.   .config  gk2_for_import.o  sample_data
..  drive    Gogo_Shogi.so     wrap_file.o


In [3]:
#!rm Gogo_Shogi.so
#!rm gk2_for_import.o
#!rm wrap_file.o
#!ls 
#!echo
#!gcc -fPIC -c -o gk2_for_import.o gk2_for_import.c -lpython3.8 -I/usr/local/lib/python3.8/dist-packages/tensorflow/include/external/local_config_python/python_include
#!gcc -fPIC -c -o wrap_file.o wrap_file.c -lpython3.8 -I/usr/local/lib/python3.8/dist-packages/tensorflow/include/external/local_config_python/python_include
#!gcc -fPIC -lpython3.8 -shared -o Gogo_Shogi.so gk2_for_import.o wrap_file.o 
#!ls -a

In [4]:
%%file test3.c
#include <stdio.h>
#include "drive/MyDrive/Gogo_Shogi/gk2_for_import.c"

int main(void){
    Initialize_Game();
    printf("&&&&%s\n",Alpha_Beta(0));
    fflush(stdout);
    return 0;
}

Writing test3.c


In [5]:
%%file Forward.py
import torch
import torch.nn as nn
import pickle
import numpy as np

def One_Hottize(board, possessed, side, my_side):
  board_one_hot=torch.zeros(8,10,5,5)
  turnIndex=side*2+my_side
  for i in range(25):
    j=2*i
    if board[j]=="F":
      koma_num=0
    if board[j]=="G" or board[j]=="O":
      koma_num=1
    if board[j]=="K":
      koma_num=2
    if board[j]=="S":
      koma_num=3
    if board[j]=="H":
      koma_num=4
    if board[j]=="A":
      koma_num=5
    if board[j]=="T":
      koma_num=6
    if board[j]=="Z":
      koma_num=7
    if board[j]=="R":
      koma_num=8
    if board[j]=="M":
      koma_num=9

    if board[j+1]=="U":
      board_one_hot[turnIndex*2+0][koma_num][int(i/5)][i%5]=1
    if board[j+1]=="D":
      board_one_hot[turnIndex*2+1][koma_num][int(i/5)][i%5]=1 
  
  possessed_num=torch.zeros(8,5)
  for koma in possessed[:12]:
    if koma=="x": break
    else:
      if koma=="F": possessed_num[turnIndex*2+0][0]+=1
      if koma=="K": possessed_num[turnIndex*2+0][1]+=1
      if koma=="S": possessed_num[turnIndex*2+0][2]+=1
      if koma=="A": possessed_num[turnIndex*2+0][3]+=1
      if koma=="H": possessed_num[turnIndex*2+0][4]+=1
  for koma in possessed[12:]:
    if koma=="x": break
    else:
      if koma=="F": possessed_num[turnIndex*2+1][0]+=1
      if koma=="K": possessed_num[turnIndex*2+1][1]+=1
      if koma=="S": possessed_num[turnIndex*2+1][2]+=1
      if koma=="A": possessed_num[turnIndex*2+1][3]+=1
      if koma=="H": possessed_num[turnIndex*2+1][4]+=1

  return [board_one_hot, possessed_num]

def forward(board, possessed, side, my_side):
  config={}
  config["side"]=0
  if my_side==0:
    with open("testmodel_m0", "rb") as f:
      model=pickle.load(f)
  if my_side==1:
    with open("testmodel_m1", "rb") as f:
      model=pickle.load(f)
  with open("config_AI", "rb") as f:
      config=pickle.load(f)
  board_t, possess_t=One_Hottize(board,possessed,side,my_side)
  value=model.forward(board_t, possess_t)
  val=value.item()
  if config["side"]!=my_side:
    val+=np.random.rand()*30
  return int(val)

Writing Forward.py


In [6]:
!gcc test3.c -o a.out -lpython3.8 -I/usr/local/lib/python3.8/dist-packages/tensorflow/include/external/local_config_python/python_include

In [7]:
!diff drive/MyDrive/Gogo_Shogi/gk2_for_import.c drive/MyDrive/Gogo_Shogi/gk2_for_import.c_temp

263,264c263,264
<     int i, j, gx=-1, gy;
< 
---
>     int i, j, gx, gy;
>     
277c277
<     if(gx==-1){ return -1;}
---
>     
297d296
< 
329d327
<     if(isOte(side)) return 0;
373,382d370
<                 if(Process_Sen_Nichi_Te(0,1-side)+1>=4){
<                     //とった手が千日手
<                     if(Remove_Sen_Nichite){
<                         //自分が先手なら千日手で負けてしまう
<                         if(side==0){Tsumiflag=1;}
<                     }
<                     //王手千日手
<                     if(isOte(side)){Tsumiflag=1;}
<                 }
<                 
396a385,393
>                     if(Process_Sen_Nichi_Te(0,1-side)+1>=4){
>                         //とった手が千日手
>                         if(Remove_Sen_Nichite){
>                             //自分が先手なら千日手で負けてしまう
>                             if(side==0){Tsumiflag=0; continue;}
>                         }
>                         //王手千日手
>                         if(isOte(side)){Tsumiflag=0; continue;}
>                   

In [8]:
#!./a.out
#!echo #?

In [9]:
%load_ext autoreload
%autoreload 2
import Gogo_Shogi as gs
import importlib
import sys


In [10]:
import json
import logging
import time

def message_flush(display=True):
  logged_flag=0
  lines=[]
  loop_count=0
  while logged_flag<2:
    logged_flag=0
    loop_count+=1
    lines=[]
    try:
      with open('/var/colab/app.log', 'r') as f:
        for line in f:
          dec_line="["+line+"]"
          j=json.loads(dec_line)
          level=j[0]['level']
          chara_list=["盤", "後", "持", "-", "\n", "5", "4", "3", "2", "1", "先"]
          if level==40 and (j[0]['msg'][0] in chara_list): 
            lines.append(j[0]['msg'])
            logged_flag+=1
          elif "A　　B　　C　　D　　E" in j[0]['msg']:
            lines.append(j[0]['msg'])
            logged_flag+=1
    except:
      print("cannot display buffer error")
      return
    if loop_count>100:
      print("cannot display buffer error")
      return
    time.sleep(0.1)
  for line in lines:
    if display: print(line)
  with open('/var/colab/app.log', 'w') as f:
      f.write("")

In [11]:
def init_game(flush=True):
  gs.Initialize_Game()
  display_game(0, flush)

def next_game_step(side, order, flush=True):
  nextside=gs.Game_Proceed(side, order)
  if nextside<=1: display_game(1-side, flush)
  return nextside

def display_game(side, flush=True):
  gs.Display_Board(side)
  message_flush(display=flush)

In [12]:
init_game(flush=True)


盤面:同じ盤面は1回目
後手: 王手していない 王手されていない 
持ち駒(後手):
------------------------------------
　 A　　B　　C　　D　　E
5　飛↓　角↓　銀↓　金↓　玉↓　
4　　 　　 　　 　　 　歩↓　
3　　 　　 　　 　　 　　 　
2　歩↑　　 　　 　　 　　 　
1　王↑　金↑　銀↑　角↑　飛↑　
------------------------------------
持ち駒(先手):
先手: 王手していない　王手されていない 詰んでいない
先手の手番:


In [13]:
import numpy as np
import torch



def One_Hottize(board, possessed, side, my_side):
  board_one_hot=torch.zeros(8,10,5,5)
  turnIndex=side*2+my_side
  for i in range(25):
    j=2*i
    if board[j]=="F":
      koma_num=0
    if board[j]=="G" or board[j]=="O":
      koma_num=1
    if board[j]=="K":
      koma_num=2
    if board[j]=="S":
      koma_num=3
    if board[j]=="H":
      koma_num=4
    if board[j]=="A":
      koma_num=5
    if board[j]=="T":
      koma_num=6
    if board[j]=="Z":
      koma_num=7
    if board[j]=="R":
      koma_num=8
    if board[j]=="M":
      koma_num=9

    if board[j+1]=="U":
      board_one_hot[turnIndex*2+0][koma_num][int(i/5)][i%5]=1
    if board[j+1]=="D":
      board_one_hot[turnIndex*2+1][koma_num][int(i/5)][i%5]=1 
  
  possessed_num=torch.zeros(8,5)
  for koma in possessed[:12]:
    if koma=="x": break
    else:
      if koma=="F": possessed_num[turnIndex*2+0][0]+=1
      if koma=="K": possessed_num[turnIndex*2+0][1]+=1
      if koma=="S": possessed_num[turnIndex*2+0][2]+=1
      if koma=="A": possessed_num[turnIndex*2+0][3]+=1
      if koma=="H": possessed_num[turnIndex*2+0][4]+=1
  for koma in possessed[12:]:
    if koma=="x": break
    else:
      if koma=="F": possessed_num[turnIndex*2+1][0]+=1
      if koma=="K": possessed_num[turnIndex*2+1][1]+=1
      if koma=="S": possessed_num[turnIndex*2+1][2]+=1
      if koma=="A": possessed_num[turnIndex*2+1][3]+=1
      if koma=="H": possessed_num[turnIndex*2+1][4]+=1

  return [board_one_hot, possessed_num]


In [14]:
import torch
import torch.nn as nn

class EvaluateNet(nn.Module):
  def __init__(self, score_sup=500, score_inf=0):
    super(EvaluateNet, self).__init__()
    self.flatten1=nn.Flatten(0,3)
    self.fc1_1=nn.Linear(2000,500)
    self.relu1_1=nn.ReLU()
    self.fc2_1=nn.Linear(500,500)

    self.flatten2=nn.Flatten(0,1)
    self.fc1_2=nn.Linear(40,100)
    self.relu1_2=nn.ReLU()
    self.fc2_2=nn.Linear(100,100)

    self.dr1=nn.Dropout(0.7)

    self.relu2=nn.ReLU()
    self.fc3=nn.Linear(600,400)
    self.relu3=nn.ReLU()
    self.fc4=nn.Linear(400,100)
    self.relu4=nn.ReLU()
    self.fc5=nn.Linear(100,1)
    self.sig=nn.Sigmoid()

    self.score_sup=score_sup
    self.score_inf=score_inf
  
  def forward(self, board, possessed):
    x1=self.flatten1(board)
    x2=self.fc1_1(x1)
    x3=self.relu1_1(x2)
    x4=self.fc2_1(x3)

    y1=self.flatten2(possessed)
    y2=self.fc1_2(y1)
    y3=self.relu1_2(y2)
    y4=self.fc2_2(y3)

    z=torch.cat((x4,y4), 0)
    z=self.dr1(z)

    z2=self.relu2(z)
    z3=self.fc3(z2)
    z4=self.relu3(z3)
    z5=self.fc4(z4)
    z6=self.relu4(z5)
    z7=self.fc5(z6)
    z8=self.sig(z7)
    f=z8*(self.score_sup-self.score_inf)+self.score_inf

    return f

In [15]:
def Evaluate_Func(model, side, my_side):
  info=gs.Get_Info()
  board_t, possess_t=One_Hottize(info[:50],info[50:],side,my_side)
  return model.forward(board_t.cuda(), possess_t.cuda())

In [16]:
import pickle
def Winning_Rate_Evaluate(n_games=100):
  random_model=EvaluateNet()
  config={}
  count=np.array([0]*4)
  for game in range(n_games):
      if np.random.randn()<0:
        my_side=0
        with open("testmodel_m1", "rb") as f:
          model_temp=pickle.load(f)
        with open("testmodel_m1", 'wb') as f:
          pickle.dump(random_model, f)
      else:
        my_side=1
        with open("testmodel_m0", "rb") as f:
          model_temp=pickle.load(f)
        with open("testmodel_m0", 'wb') as f:
          pickle.dump(random_model, f)
      config["side"]=my_side
      with open('config_AI', 'wb') as f:
        pickle.dump(config, f)
      side=0
      init_game(flush=False)
      for step in range(150):
        result=next_game_step(0,gs.Alpha_Beta(0), flush=False)
        side=1
        display_game(1, flush=False)
        if result>=2: 
          if my_side==1:
            result=7-result
          if result==2:
            result=5
          break
        result=next_game_step(1,gs.Alpha_Beta(1), flush=False)
        side=0
        display_game(0, flush=False)
        if result>=2:
          if my_side==0:
            result=7-result
          if result==2:
            result=5
          break
      #my-sideが勝ったら3、負けたら4、引き分けたら5、千日手で勝ったら6
      if result==3:
        count[0]+=1
      elif result==6:
        count[1]+=1
      elif result==4:
        count[2]+=1
      else:
        count[3]+=1
      if my_side==0:
        with open("testmodel_m1", 'wb') as f:
          pickle.dump(model_temp, f)
      else:
        with open("testmodel_m0", "wb") as f:
          pickle.dump(model_temp, f)
  return count/n_games

In [17]:
#Winning_Rate_Evaluate(10)

In [18]:
import pickle
import numpy as np
import time
from google.colab import files
def model_train(n_epochs, file_read=True, n_models=2, n_games=200, filename="testmodel", gamma=1, s_rate=0.01, ev_interval=10):
  models=[]
  config={}
  for i in range(n_models):
    models.append(EvaluateNet())
  if file_read:
    with open("drive/MyDrive/Gogo_Shogi/testmodel_m0", "rb") as f:
          models[0]=pickle.load(f)
    with open("drive/MyDrive/Gogo_Shogi/testmodel_m1", "rb") as f:
          models[1]=pickle.load(f) 
  model0=models[np.random.randint(0,n_models)]
  model1=models[np.random.randint(0,n_models)]
  s_count=0
  with open(filename+'_m0', 'wb') as f:
    pickle.dump(model0.cpu(), f)
  with open(filename+'_m1', 'wb') as f:
    pickle.dump(model1.cpu(), f)
  for epoch in range(n_epochs):
    count=0
    print("Epoch "+str(epoch))
    for game in range(n_games):
      print("Game "+str(game)+":")
      if np.random.randn()<0:
        my_side=0
      else:
        my_side=1
      config["side"]=my_side
      with open('config_AI', 'wb') as f:
        pickle.dump(config, f)
      side=0
      model0=models[np.random.randint(0,n_models)]
      model1=models[np.random.randint(0,n_models)]
      opt0 = torch.optim.Adam(model0.parameters(), lr=0.01)
      opt1 = torch.optim.Adam(model1.parameters(), lr=0.01)
      init_game(flush=False)
      if my_side==0:
        model0.cuda()
        s=Evaluate_Func(model0, side, my_side)
      if my_side==1:
        model1.cuda()
        s=Evaluate_Func(model1, side, my_side)
      side=0
      for step in range(150):
        display_game(0)
        result=next_game_step(0,gs.Alpha_Beta(0), flush=False)
        if side==my_side: s=s*gamma+Evaluate_Func(model0, side, my_side)
        side=1
        if result>=2: 
          if my_side==1:
            result=7-result
          if result==2:
            result=5
          break
        display_game(1)
        result=next_game_step(1,gs.Alpha_Beta(1), flush=False)
        if side==my_side: s=s*gamma+Evaluate_Func(model1, side, my_side)
        side=0
        if result>=2:
          if my_side==0:
            result=7-result
          if result==2:
            result=5
          break
      #my-sideが勝ったら3、負けたら4、引き分けたら5
      print("n_step:"+str(step))
      display_game(side)
      if result==3:
        count+=1
        s=-s
      if result==6:
        s_count+=1
      if result==5:
        continue
      s.backward()
      if my_side==0:
        opt0.step()
      if my_side==1:
        opt1.step()
      print("")
    print("Learning Winning Rate:"+str(count/n_games))
    #time.sleep(5)
    with open(filename+'_m0', 'wb') as f:
      pickle.dump(model0.cpu(), f)
    with open(filename+'_m1', 'wb') as f:
      pickle.dump(model1.cpu(), f)
    with open('drive/MyDrive/Gogo_Shogi/'+filename+'_m0', 'wb') as f:
      pickle.dump(models[0].cpu(), f)
    with open('drive/MyDrive/Gogo_Shogi/'+filename+'_m1', 'wb') as f:
      pickle.dump(models[1].cpu(), f)
    if (epoch+1)%ev_interval==0:
      rate=Winning_Rate_Evaluate()
      print("Epoch: ", epoch, "Evaluated(pure-win, sen-nichi-te-win, lose, draw): ", rate)
      with open('drive/MyDrive/Gogo_Shogi/eval_log.txt', 'a') as f:
        print("Timestamp",time.time(),"Epoch: ", epoch, "Evaluated(pure-win, sen-nichi-te-win, lose, draw):", rate, file=f)
      time.sleep(5)
    #if s_count>=3:
    #  s_count=0
    #  del model2
    #  model2=EvaluateNet()


In [19]:
model=EvaluateNet()
Evaluate_Func(model.cuda(), 0,0).item()

243.14869689941406

In [ ]:
model_train(2000,n_models=10, n_games=1, filename="testmodel", ev_interval=100)

ストリーミング出力は最後の 5000 行に切り捨てられました。
盤面:同じ盤面は1回目
後手: 王手していない 王手されていない 詰んでいない
持ち駒(後手):
------------------------------------
　 A　　B　　C　　D　　E
5　　 　　 　銀↓　金↓　玉↓　
4　飛↓　　 　角↓　　 　歩↓　
3　　 　　 　　 　　 　　 　
2　歩↑　　 　角↑　銀↑　　 　
1　王↑　金↑　　 　　 　飛↑　
------------------------------------
持ち駒(先手):
先手: 王手していない　王手されていない 
後手の手番:
盤面:同じ盤面は1回目
後手: 王手していない 王手されていない 
持ち駒(後手):
------------------------------------
　 A　　B　　C　　D　　E
5　　 　　 　　 　金↓　玉↓　
4　飛↓　銀↓　角↓　　 　歩↓　
3　　 　　 　　 　　 　　 　
2　歩↑　　 　角↑　銀↑　　 　
1　王↑　金↑　　 　　 　飛↑　
------------------------------------
持ち駒(先手):
先手: 王手していない　王手されていない 詰んでいない
先手の手番:
------------------------------------
持ち駒(先手):
先手: 王手していない　王手されていない 
後手の手番:
盤面:同じ盤面は1回目
後手: 王手していない 王手されていない 
持ち駒(後手):
------------------------------------
　 A　　B　　C　　D　　E
5　　 　　 　銀↓　金↓　玉↓　
4　飛↓　　 　角↓　　 　歩↓　
3　　 　　 　　 　角↑　　 　
2　歩↑　　 　　 　銀↑　　 　
1　王↑　金↑　　 　　 　飛↑　
------------------------------------
持ち駒(先手):
先手: 王手していない　王手されていない 詰んでいない
先手の手番:
盤面:同じ盤面は1回目
後手: 王手していない 王手されていない 詰んでいない
持ち駒(後手):
------------------------------------
　 A　　B

In [ ]:
!cat drive/MyDrive/Gogo_Shogi/eval_log.txt

In [ ]:
model1.state_dict()

OrderedDict([('fc1_1.weight',
              tensor([[ 0.0080,  0.0169,  0.0126,  ...,  0.0085,  0.0113, -0.0131],
                      [-0.0101, -0.0063, -0.0003,  ..., -0.0183, -0.0049, -0.0052],
                      [ 0.0088,  0.0035, -0.0104,  ..., -0.0190, -0.0154,  0.0076],
                      ...,
                      [-0.0045,  0.0061,  0.0166,  ..., -0.0074,  0.0201,  0.0129],
                      [ 0.0061,  0.0056,  0.0176,  ...,  0.0051,  0.0091, -0.0145],
                      [-0.0112,  0.0122,  0.0081,  ...,  0.0124, -0.0026, -0.0221]])),
             ('fc1_1.bias',
              tensor([-1.6795e-01, -1.4820e-01,  2.0234e-02, -6.9655e-03,  1.5316e-01,
                       1.8547e-02,  1.6840e-01, -1.4908e-01,  1.0489e-02, -1.4111e-01,
                       3.5432e-04, -1.4184e-01, -1.5076e-01, -1.8041e-01,  4.7898e-03,
                      -2.5163e-03, -5.0648e-03, -1.5915e-03, -8.5230e-03, -1.5602e-01,
                       1.2812e-02,  1.2663e-02,  5.1904e-04,

In [ ]:
for step in range(150):
    result=next_game_step(0,gs.Alpha_Beta(0), flush=False)
    if result==3:
        print("You Lose")
        break
    if result==4:
        print("You Win")
        break
    if result==5:
        print("Draw")
        break
    order=input()
    result=next_game_step(1,order)
    if result==3:
        print("You Win")
        break
    if result==4:
        print("You Lose")
        break
    if result==5:
        print("Draw")
        break

In [ ]:
init_game()
t1, t2=One_Hottize(gs.Get_Info()[:50],gs.Get_Info()[50:],1,0)
s1=model.forward(t1, t2)
next_game_step(0,"1E4E")
t1, t2=One_Hottize(gs.Get_Info()[:50],gs.Get_Info()[50:],1,0)
s2=model.forward(t1, t2)
s3=s1+s2*0.3
print(s1)
print(s2)
print(s3)
s3.backward()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

optimizer.step()

1E4E
盤面:同じ盤面は1回目
後手: 王手していない 王手されている 詰んでいない
持ち駒(後手):
------------------------------------
　 A　　B　　C　　D　　E
5　飛↓　角↓　銀↓　金↓　玉↓　
4　　 　　 　　 　　 　飛↑　
3　　 　　 　　 　　 　　 　
2　歩↑　　 　　 　　 　　 　
1　王↑　金↑　銀↑　角↑　　 　
------------------------------------
持ち駒(先手):歩　
先手: 王手している　王手されていない 
後手の手番:
1E4E
盤面:同じ盤面は1回目
後手: 王手していない 王手されている 詰んでいない
持ち駒(後手):
------------------------------------
　 A　　B　　C　　D　　E
5　飛↓　角↓　銀↓　金↓　玉↓　
4　　 　　 　　 　　 　飛↑　
3　　 　　 　　 　　 　　 　
2　歩↑　　 　　 　　 　　 　
1　王↑　金↑　銀↑　角↑　　 　
------------------------------------
持ち駒(先手):歩　
先手: 王手している　王手されていない 
後手の手番:
tensor([48.0274], grad_fn=<AddBackward0>)
tensor([48.0353], grad_fn=<AddBackward0>)
tensor([62.4380], grad_fn=<AddBackward0>)


In [ ]:
#sample play
init_game()
side=0
while side<=1:
  order=input()
  side=next_game_step(side, order)
  if side==3:
    print("You Win")
  if side==4:
    print("You Lose")
  if side==5:
    print("Draw") 

盤面:同じ盤面は1回目
後手: 王手していない 王手されていない 
持ち駒(後手):
------------------------------------
　 A　　B　　C　　D　　E
5　飛↓　角↓　銀↓　金↓　玉↓　
4　　 　　 　　 　　 　歩↓　
3　　 　　 　　 　　 　　 　
2　歩↑　　 　　 　　 　　 　
1　王↑　金↑　銀↑　角↑　飛↑　
------------------------------------
持ち駒(先手):
先手: 王手していない　王手されていない 詰んでいない
先手の手番:
!w!w
You Lose


In [ ]:
#The Following is something like a test.

In [ ]:
%%file test1.c
#include <stdio.h>

void he(void){
    printf("HELLOOOOOOOOOO!");
    return;
}

Writing test1.c


In [ ]:
from google.colab import files
#files.download('Gogo_Shogi.so')

In [ ]:
%%file test2.c
#define PY_SSIZE_T_CLEAN
#include "/usr/local/lib/python3.8/dist-packages/tensorflow/include/external/local_config_python/python_include/Python.h"
#include <stdlib.h>
extern void he(void);

PyObject* f_he(PyObject* self, PyObject* args)
{
  if (!PyArg_ParseTuple(args, ""))
    return NULL;
  he();
  return Py_BuildValue("");
}

static PyMethodDef methods[] = {
    {"He", f_he, METH_VARARGS},
    {NULL}
};


static struct PyModuleDef Test_Module =
{
    PyModuleDef_HEAD_INIT,
    "Test_Module",  
    "",
    -1,
    methods
};

PyMODINIT_FUNC PyInit_Test_Module(void)  {
    return PyModule_Create(&Test_Module);
}

Writing test2.c


In [ ]:
!gcc -fPIC -c -o test1.o test1.c
!gcc -fPIC -c -o test2.o test2.c -I/usr/local/lib/python3.8/dist-packages/tensorflow/include/external/local_config_python/python_include
!gcc -fPIC -shared -o Test_Module.so test1.o test2.o 
!ls -a

.	 gk2_for_import.c    logconf	   test1.c  Test_Module.so
..	 gk2_for_import.o    log.conf	   test1.o  wrap_file.c
app.log  Gogo_Shogi.so	     rotation.log  test2.c  wrap_file.o
.config  .ipynb_checkpoints  sample_data   test2.o


In [ ]:
import Test_Module as tm

In [ ]:
tm.He()